In [1]:
import torch
import torch.nn as nn
import torch.optim as optim6
import matplotlib.pyplot as plt
from torch.autograd import Variable # storing data while learning
from config import CFG ,Args
from baselineUtils import load_datasets,distance_metrics
from utils import ScheduledOptim,visualize_preds
from train import train_attn_mdn 
from test import test_mdn 
from torch.optim.lr_scheduler import LambdaLR 
from model import Attention_GMM #,Attention_GMM_Encoder,Transformer_MDN
# from torch.utils.data.distributed import  DistributedSampler
# from torch.nn.parallel import DistributedDataParallel as DDP
# from torch.distributed import init_process_group,destroy_process_group
import os

/home/avl1/anaconda3/envs/mdn/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Device used and Data loader

In [2]:
device = CFG.device
batch_size = CFG.batch_size
print(f"Using {device} device")
args = Args 

Using cuda:1 device


In [3]:
train_dataset, val_dataset,test_dataset,mean,std = load_datasets(args)

start loading dataset
validation set size -> 0
001 / 007 - loading crowds_zara01_train.txt
002 / 007 - loading crowds_zara03_train.txt
003 / 007 - loading students003_train.txt
004 / 007 - loading students001_train.txt
005 / 007 - loading uni_examples_train.txt
006 / 007 - loading biwi_eth_train.txt
007 / 007 - loading biwi_hotel_train.txt
start loading dataset
validation set size -> 0
001 / 007 - loading biwi_eth_val.txt
002 / 007 - loading students001_val.txt
003 / 007 - loading uni_examples_val.txt
004 / 007 - loading crowds_zara03_val.txt
005 / 007 - loading crowds_zara01_val.txt
006 / 007 - loading biwi_hotel_val.txt
007 / 007 - loading students003_val.txt
start loading dataset
validation set size -> 0
001 / 001 - loading crowds_zara02.txt


In [4]:
train_dl = torch.utils.data.DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True, num_workers=CFG.num_workers)
val_dl = torch.utils.data.DataLoader(val_dataset, batch_size=CFG.batch_size, shuffle=True, num_workers=CFG.num_workers)
test_dl = torch.utils.data.DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=CFG.num_workers)

In [5]:
# Train
in_features = CFG.in_features
out_features = CFG.out_features
num_heads = CFG.num_heads
num_encoder_layers = CFG.num_encoder_layers
num_decoder_layers =  CFG.num_decoder_layers
embedding_size = CFG.embd_size
max_length = 8
n_hidden = CFG.n_hidden
gaussians = CFG.gaussians
forecast_window = 12
drp = CFG.drop_out
add_features = CFG.add_features

In [6]:
# If you want to train transformer only copy it from commented.py
# Train attention MDN
attn_mdn = Attention_GMM(device,in_features,out_features,num_heads,num_encoder_layers,num_decoder_layers,embedding_size,n_gaussians=gaussians,n_hidden = n_hidden, dropout=drp).to(device)
#attn_mdn = Attention_GMM_Encoder(device,in_features,out_features,num_heads,num_encoder_layers,num_decoder_layers,embedding_size,n_gaussians=gaussians,n_hidden = n_hidden, dropout=drp).to(device)
# if torch.cuda.device_count() > 1:
#     print("Using", torch.cuda.device_count(), "GPUs")
#     attn_mdn = DDP(attn_mdn,device_ids=[0,1])
for p in attn_mdn.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
# Define the optimizer
optimizer = ScheduledOptim(
        torch.optim.Adam(attn_mdn.parameters(), betas=(0.9, 0.98), eps=1e-09),
        CFG.lr_mul, CFG.d_model, CFG.n_warmup_steps) #len(train_dl)True
#         print(name, child)

In [7]:
if(Args.mode=='test'):
    PATH = Args.model_path
    attn_mdn = torch.load(PATH).to(device)
else:
    loss_train, loss_eval,val_mad,val_fad = train_attn_mdn(train_dl,val_dl,test_dl,attn_mdn,optimizer,add_features,mixtures =gaussians, epochs=CFG.epochs,mean=mean,std=std)

In [8]:
if(Args.mode=='train'):
    fig = plt.figure(1)	#identifies the figure 
    plt.title(" Training Loss Per Epoch", fontsize='16')	#title
    plt.plot(loss_train,color='Blue',label='Training Loss')	#plot the points
    plt.plot(loss_eval,color='Green',label='Evaluation Loss')	#plot the points
    plt.legend(loc="upper right")
    plt.show()

In [9]:
if(Args.mode=='train'):
    fig = plt.figure(2)	#identifies the figure 
    plt.title("Evaluation Error", fontsize='16')	#title
    # plt.plot(test_mad,color='Green', label="ADE Test")	#plot the points
    # plt.plot(test_fad,color='Red', label="FDE Test")	#plot the points
    plt.plot(val_mad,color='Green', label="ADE Validation")	#plot the points
    plt.plot(val_fad,color='Red', label="FDE Validation")	#plot the points
    plt.legend(loc="upper right")
    plt.show()

In [11]:
# Test the model
batch_preds,batch_gts,avg_mad,avg_fad,candidate_trajs,candidate_weights,best_candiates,src_trajs = test_mdn(test_dl, attn_mdn,device,add_features = add_features,mixtures=gaussians,enc_seq = 8,dec_seq=12, mode='feed',loss_mode ='mdn',mean=mean,std=std)


Batch: 60 / 60: 100%|██████████| 60/60 [00:06<00:00,  9.43it/s]

Test loss: -2.225660828019803
Test loss with msq: -1.0202178960045178
Test avg_mad: 0.3191342531281795
Test avg_fad: 0.7018245713246919
Test avg_mad_best: 0.3198853881146412
Test avg_fad_best: 0.7181163657346215
Test avg_acc_mad: [0.32833922]
Test avg_acc_fad: [0.72632706]


shape of candidate trajs (num_batchs, bacth_size, x, 2, 12, 2)  ---> where x is the number of candidate trajectories
shape of ground truth (num_batchs, bacth_size, 12, 2) ---> 

In [12]:
# Visulaize output via visualize_preds
if Args.visualize:
    visualize_preds(src_trajs,batch_gts,candidate_trajs)